### In this final assignment your goal is to process a large corpus of unstructured chat data, discover it's structure and build a Question Answering system based on it

In [1]:
import re
import os
import json
import nltk
import scipy
import pymystem3
import numpy as np
import pandas as pd

from nltk.tag import pos_tag
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models import KeyedVectors
from itertools import chain
from tqdm import tqdm

C:\Anaconda3\envs\theano\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


### 3.1 Unzip and explore the data

In [3]:
# !unzip ./support_logs.zip

In [7]:
faq = open("./support_faq.txt", encoding='utf8').read()

In [8]:
raw_chats = open("./support_chats.txt", encoding='utf8').read()

In [9]:
print(faq[:995])

1 . Могу ли я поменять ПИН-код своей карты <bankname> Банка в банкомате ?
Да , если банкомат поддерживает данную функцию .

2 . В каких валютах можно расплачиваться картой <bankname> Банка ?
Расплачиваться картой и снимать наличные можно в любой валюте .

3 . При оплате товаров за границей , я оплачиваю НДС в казну другого государства , возможно ли вернуть эти средства ?
Да , возможно . Для этого вам необходимо в магазине , в котором вы приобретаете товар , попросить предоставить дополнительный чек Tax Free . На данный чек необходимо поставить штамп на таможне . Возврат средств по чекам Tax Free возможен в международном аэропорту ( в стране , где совершалась покупка ) в специальном пункте возврата НДС .

4 . Безопасно ли направлять ссылку на скачивание выписки на сайте банка ? Может ли другой человек получить мою выписку на сайте банка ?
Получать выписку по ссылке в письме так же безопасно , как получать выписку в виде вложения - ссылка уникальна и генерируется случайным образом .


In [12]:
print(raw_chats[:1000])

Chat number = 229391
Client Id = 4569861

20:31:24 Клиент : Добрый день ! Могу ли я частично изъять со счета денежные средства ранее 1 месяцев ?
20:32:33 Сотрудник : Здравствуйте !
20:33:15 Сотрудник : Только полностью закрыть вклад .
20:33:34 Клиент : Сколько будет стоить перевод со своей карты в размере 111 т.р . на две карты клиентам <bankname> банк ?
-- -- -- --

Chat number = 229390
Client Id = 1522643

20:30:51 Клиент : Здравствуйте !
20:31:38 Клиент : Подскажите , где в онлайн-банке партнерские спецпредложения по картам ?
20:32:15 Сотрудник : Здравствуйте !
20:32:58 Сотрудник : К сожалению , в новой версии этого раздела еще нет .
20:33:20 Клиент : Ясно , будем ждать . Всего доброго !
20:33:26 Сотрудник : Всего Вам доброго ! Будут еще вопросы , обращайтесь !
-- -- -- --

Chat number = 229389
Client Id = 4680324

20:30:46 Клиент : Добрый вечер
20:31:23 Сотрудник : Здравствуйте !
20:32:04 Сотрудник : Чем я могу Вам помочь ?
20:32:46 Клиент : У меня была операция по привязке карты к

In [27]:
raw_chats.split('-- -- -- --')[0].split('\n')

['Chat number = 229391',
 'Client Id = 4569861',
 '',
 '20:31:24 Клиент : Добрый день ! Могу ли я частично изъять со счета денежные средства ранее 1 месяцев ?',
 '20:32:33 Сотрудник : Здравствуйте !',
 '20:33:15 Сотрудник : Только полностью закрыть вклад .',
 '20:33:34 Клиент : Сколько будет стоить перевод со своей карты в размере 111 т.р . на две карты клиентам <bankname> банк ?',
 '']

In [41]:
data = []
for chat in tqdm(raw_chats.split('-- -- -- --')):
    client = 0
    personal = 0
    chat_id = -1
    client_id = -1
    med_data = {'client_reply':[], 'personal_reply':[]}
    for string in chat.split('\n'):
        try:
            if string.find('Chat number') != -1:
                chat_id = string.split('=')[1].strip()
            elif string.find('Client Id') != -1:
                client_id = string.split('=')[1].strip()
            elif string.find('Клиент') != -1:
                med_data['client_reply'].append([string.split('Клиент :')[1].strip(), client, personal])
                client += 1
            elif string.find('Сотрудник') != -1:
                med_data['personal_reply'].append([string.split('Сотрудник :')[1].strip(), client, personal])
                personal += 1
        except:
            pass
        
    data.append([client_id, chat_id, med_data])

100%|█████████████████████████████████████████████████████████████████████████| 28363/28363 [00:02<00:00, 11531.85it/s]


In [43]:
data

[['4569861',
  '229391',
  {'client_reply': [['Добрый день ! Могу ли я частично изъять со счета денежные средства ранее 1 месяцев ?',
     0,
     0],
    ['Сколько будет стоить перевод со своей карты в размере 111 т.р . на две карты клиентам <bankname> банк ?',
     1,
     2]],
   'personal_reply': [['Здравствуйте !', 1, 0],
    ['Только полностью закрыть вклад .', 1, 1]]}],
 ['1522643',
  '229390',
  {'client_reply': [['Здравствуйте !', 0, 0],
    ['Подскажите , где в онлайн-банке партнерские спецпредложения по картам ?',
     1,
     0],
    ['Ясно , будем ждать . Всего доброго !', 2, 2]],
   'personal_reply': [['Здравствуйте !', 2, 0],
    ['К сожалению , в новой версии этого раздела еще нет .', 2, 1],
    ['Всего Вам доброго ! Будут еще вопросы , обращайтесь !', 3, 2]]}],
 ['4680324',
  '229389',
  {'client_reply': [['Добрый вечер', 0, 0],
    ['У меня была операция по привязке карты кукуруза , она задвоилась , и с меня три раза сняли комиссию в 11р .',
     1,
     2],
    ['Ваш

In [19]:
len(raw_chats.split('-- -- -- --'))

28363

### The data consists of 2 files. The first one contains raw logs of some bank's customer support. The second one contains frequently asked questions with answers. 

### 3.2 Building a structured FAQ

### A good place to start would be to turn the FAQ into a more structured form, then find paraphrases for each question from the chat logs.

In [42]:
faq_qna = []
buf = []
for line in faq.split("\n"):
    if line == '' and len(buf):
        faq_qna.append(buf)
        buf = []
    else:
        buf.append(line)
        
FAQ = []
for faq_entry in faq_qna:
    faq_dict = {}
    faq_dict['answer'] = faq_entry[1]
    faq_dict['question'] = faq_entry[0]
    faq_dict['paraphrased_questions'] = []
    FAQ.append(faq_dict)

In [46]:
FAQ

[{'answer': 'Да , если банкомат поддерживает данную функцию .',
  'paraphrased_questions': [],
  'question': '1 . Могу ли я поменять ПИН-код своей карты <bankname> Банка в банкомате ?'},
 {'answer': 'Расплачиваться картой и снимать наличные можно в любой валюте .',
  'paraphrased_questions': [],
  'question': '2 . В каких валютах можно расплачиваться картой <bankname> Банка ?'},
 {'answer': 'Да , возможно . Для этого вам необходимо в магазине , в котором вы приобретаете товар , попросить предоставить дополнительный чек Tax Free . На данный чек необходимо поставить штамп на таможне . Возврат средств по чекам Tax Free возможен в международном аэропорту ( в стране , где совершалась покупка ) в специальном пункте возврата НДС .',
  'paraphrased_questions': [],
  'question': '3 . При оплате товаров за границей , я оплачиваю НДС в казну другого государства , возможно ли вернуть эти средства ?'},
 {'answer': 'Получать выписку по ссылке в письме так же безопасно , как получать выписку в виде в

### From now on, you are on your own. 

![architecture](showme.png)

### 3.3 Find paraphrases for existing questions

### 3.4 Find clusters of frequent questions